In [1]:
from augment import *
import glob
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                               #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from GAN import *                  
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #

In [2]:
def dstnc(dt2,dt1): #فاصله و واریانس بردار فاصله ی هر کدام از ردیف های دیتاست(آگمنت)1 از (کل)2
    dist,vrnc,i,h,w=[],[],0,np.shape(dt1)[0],np.shape(dt1)[1]
    end=min(len(dt2[0]),len(dt1[0]))
    for i in range (h):
        ds1 = dt2-dt1[i,:end] #.reshape(-1,1) #distance of every training instance
        dist_array=(ds1*10)**3  #معیار فاصله با حساسیت به توان 3 (حفظ علامت و تاثیر بیشتر تفاوت زیاد در نقاط)
        vr=np.var(ds1)
        ds1=min(np.sum(np.absolute(dist_array),axis=1))
        #print('dist2 = ', dist2)
        vrnc.append(vr)
        dist.append(ds1)
    variance=np.reshape(vrnc,(-1,1))
    distnc=np.reshape(dist,(-1,1))
    return(distnc,variance)

cls_num=2
ecg=np.array([])
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=TwoLeadECG(i)         #فراخوانی داده های اصلی
    ecg=np.append(ecg,vars()['ecg'+str(i)])
wdth=int(len(ecg1[0]))
ecg=np.reshape(ecg,(int(len(ecg)/wdth),wdth))  #base data windows
print('\nfinal_train_shap=',np.shape(ecg))
os.chdir(cwd)

first_train_shape= (23, 83)
first_test_shape= (1139, 83)
classes_quantity= 2
tr_lbls=	 {1, 2}
Count_labels= [12 11]
max(train_feature_Altitude)= 2.0
min(train_feature_Altitude)= -3.1489147
first_train_sample=
 [ 2.          0.68834554  0.6172271   0.6172271   0.56981481  0.68834554
  0.64093325  0.71205169  0.78317013  0.49869637  0.47499023  0.54610867
  0.33275335  0.33275335  0.26163491  0.23792876  0.26163491  0.21422262
  0.21422262  0.26163491  0.23792876  0.09569189  0.11939803  0.11939803
 -0.02283885  0.16681032  0.28534105  0.16681032  0.04827959  0.16681032
  0.16681032  0.14310418 -0.80514168 -2.1800981  -3.0098133  -3.0098133
 -2.8438703  -2.962401   -2.5831026  -2.0378613  -2.0852736  -1.8482121
 -1.4689138  -1.2318524  -1.1370278  -0.6629048  -0.37843104 -0.33101875
 -0.14136958  0.28534105  0.45128408  0.49869637  0.40387178  0.33275335
  0.47499023  0.28534105  0.42757794  0.49869637  0.6172271   0.54610867
  0.49869637  0.54610867  0.6172271   0.6646394   0.6646394   

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
base_wn=np.array([])
windws=np.array([])

for cls in range (1,cls_num+1):                   #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])
    base_wn=np.append(base_wn,vars()['wndws'+str(cls)])
base_wn=np.reshape(base_wn,(int(len(base_wn)/wdth),wdth))  #base data windows
print("shape wndws1=", np.shape(wndws1))
print("shape base_wn=", np.shape(base_wn))

furir_windws0=np.fft.fft(np.array(base_wn[:,:-1]))
print("shape furir_windws0=", np.shape(furir_windws0))
furir_windws=np.concatenate((np.array(furir_windws0),np.transpose([base_wn[:,-1]])),axis=1)   
print("shape furir_windws=", np.shape(furir_windws))

smpl_rte=len(base_wn[0])-1                 #len(ecg1[0])  # در ماژول ها نیز همین مقدار ثبت شده
print("smpl_rte=",smpl_rte)
mx_wndws=np.max(np.bincount(np.int16(base_wn[:,-1])))
print("mx_wndws=",mx_wndws)

shape wndws1= (12, 481)
shape base_wn= (23, 481)
shape furir_windws0= (23, 480)
shape furir_windws= (23, 481)
smpl_rte= 480
mx_wndws= 12


In [4]:
path=('{}\AugEvl'.format(cwd)) #create folder 
if not os.path.isdir(path):
    os.mkdir(path)

def addqlty(base_wn_dt,data,mthd,i):               
    cls=int(i)
    c,alpha,PL=0,1,1
    #from glob import glob
    cwd = os.getcwd() #
    adrs=cwd+'/Aug/aug'+str(i)+str(mthd)+'.csv'
    #alldata = np.array([])                    # List for storing all the data
    train_labels=np.array(base_wn_dt[:,-1])
    tr=np.array(base_wn_dt[:,:-1])
    indx_pstv=np.where(base_wn_dt[:,-1] == i)
    P=np.array(base_wn_dt[indx_pstv])
    indx_ngtv=np.where(base_wn_dt[:,-1] != i)
    N=np.array(base_wn_dt[indx_ngtv])

    #data = np.array(data)
    aug=np.array(data)
    pnts=int(np.shape(data)[1])
    
    #aug=data[data[:,-2].argsort()]        # Positive data Sorted Base Distance
    trl=np.array(train_labels)
    for j in range (len(trl)):
        if train_labels[j]==i :           # !!!!! == (Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    #mn_ds_pstv=np.reshape(aug[:,-2],(-1,1)) #min dis to positive class (submited in -2 column)
    print('shape and aug[0]=',np.shape(aug[0]),aug[0])
    print('shape P and P[0]=',np.shape(P),P[0])    
    qlty1,qlty2=mn_ds_pstv,vr_P=dstnc(P[:,1:-1],aug[:,1:-1])  #min distance to negative class
    new1=np.append(aug,qlty1,axis=1)
    new2=np.append(new1,qlty2,axis=1)
    mn_ds_ngtv,vr_N=dstnc(N[:,1:-1],aug[:,1:-1])              #min distance to negative class
    qlty3=mn_ds_pstv/mn_ds_ngtv                               #Smaller is Better
    qlty4=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new3=np.append(new2,qlty3,axis=1)
    new4=np.append(new3,qlty4,axis=1)
    for j in range (len(trl)):
        if train_labels[j]!=i :                          # !!!!! == (!= Inverse Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    qlty5=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new5=np.append(new4,qlty5,axis=1)
    print('class=',i,'shape(new5)=',np.shape(new5),end='\t')
    print('mx=',np.max(new5[:,1:-7]),end='\t')
    print('mn=',np.min(new5[:,1:-7]))
    pd.DataFrame(new5[:,:]).to_csv('{}/AugEvl/AugEvl{}{}.csv'.format(cwd,cls,mthd))
    print()

In [5]:
xtrain=np.empty((0,smpl_rte+1), float) 

for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug'+str(i)+'00']=np.empty((0,smpl_rte+1), float) #Scale    
    vars()['ecg_aug'+str(i)+'10']=np.empty((0,smpl_rte+1), float) #Scale
    vars()['ecg_aug'+str(i)+'20']=np.empty((0,smpl_rte+1), float) #Mag Warp
    vars()['ecg_aug'+str(i)+'30']=np.empty((0,smpl_rte+1), float) #TimWarp
    vars()['ecg_aug'+str(i)+'40']=np.empty((0,smpl_rte+1), float) #GDO
    vars()['ecg_aug'+str(i)+'50']=np.empty((0,smpl_rte+1), float) #frequence
    vars()['ecg_aug'+str(i)+'60']=np.empty((0,smpl_rte+1), float) #GAN
    vars()['ecg_aug'+str(i)+'61']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'62']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'70']=np.empty((0,smpl_rte+1), float) #LSTM
    vars()['ecg_aug'+str(i)+'71']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'72']=np.empty((0,smpl_rte+1), float)

'''f_scr=np.array([54,83,88,82,70,58,54,70])      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
f_scr_difrnc=f_scr-f_scr[0]
f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
aug_amnt=np.array(f_scr_rtio)                 #aug_amnt[0] is UnAg so in follow start from 1 for scale,..
'''
vlum_wndw=10*mx_wndws  

for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vlum_sig=10*mx_wndws-len(vars()['wndws'+str(i)])
    
    vars()['ecg_aug'+str(i)+'10']=np.append(vars()['ecg_aug'+str(i)+'10'],scl_rndm_beat(ecg,i,int(vlum_sig),mag=.1),axis=0)#0.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'10'],10,i)
    print(str(i)+'10 =',len(vars()['ecg_aug'+str(i)+'10']))
    
    vars()['ecg_aug'+str(i)+'20']=np.append(vars()['ecg_aug'+str(i)+'20'],MgWrp_rndm_beat(ecg,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'20'],20,i)
    print(str(i)+'20 =',len(vars()['ecg_aug'+str(i)+'20']))
    
    vars()['ecg_aug'+str(i)+'30']=np.append(vars()['ecg_aug'+str(i)+'30'],TimWrp_rndm_beat(ecg,i,int(vlum_sig),mag=.2),axis=0)#.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'30'],30,i)
    print(str(i)+'30 =',len(vars()['ecg_aug'+str(i)+'30']))
    
    vars()['ecg_aug'+str(i)+'40']=np.append(vars()['ecg_aug'+str(i)+'40'],GDO_rndm_beat(ecg,i,int(vlum_sig),mag=.1),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'40'],40,i)
    print(str(i)+'40 =',len(vars()['ecg_aug'+str(i)+'40']))
    
    vars()['ecg_aug'+str(i)+'50']=np.append(vars()['ecg_aug'+str(i)+'50'],frqnc_rndm_beat(furir_windws,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'50'],50,i)
    print(str(i)+'50 =',len(vars()['ecg_aug'+str(i)+'50']))

shape and aug[0]= (481,) [ 9.72629277e-02  1.22623485e-01  1.39908346e-01  1.48297075e-01
  1.48231539e-01  1.41340334e-01  1.30144619e-01  1.17601694e-01
  1.06578240e-01  9.93603943e-02  9.72990895e-02  1.00658271e-01
  1.08688033e-01  1.19895052e-01  1.32440662e-01  1.44572167e-01
  1.54990882e-01  1.63080808e-01  1.68958938e-01  1.73352586e-01
  1.77349432e-01  1.82092702e-01  1.88500869e-01  1.97077766e-01
  2.07849673e-01  2.20429042e-01  2.34170496e-01  2.48362407e-01
  2.62392483e-01  2.75839032e-01  2.88466439e-01  3.00135987e-01
  3.10671541e-01  3.19735634e-01  3.26770003e-01  3.31035907e-01
  3.31758497e-01  3.28345076e-01  3.20619133e-01  3.08999341e-01
  2.94559732e-01  2.78933507e-01  2.64062077e-01  2.51833292e-01
  2.43686760e-01  2.40280268e-01  2.41304037e-01  2.45499516e-01
  2.50893091e-01  2.55203483e-01  2.56337550e-01  2.52864014e-01
  2.44355273e-01  2.31514606e-01  2.16053823e-01  2.00343568e-01
  1.86911399e-01  1.77898651e-01  1.74597072e-01  1.77167988e-01



110 = 108
shape and aug[0]= (481,) [-0.49421782 -0.54736476 -0.54971952 -0.51200992 -0.45030841 -0.38239621
 -0.324086   -0.28629765 -0.27346054 -0.28347718 -0.30911266 -0.34036101
 -0.36715311 -0.38174749 -0.38027952 -0.36319228 -0.3345673  -0.30063805
 -0.2679433  -0.2416228  -0.2242746  -0.21561046 -0.21291859 -0.21213135
 -0.20915351 -0.20106558 -0.18688083 -0.16767926 -0.14612451 -0.125539
 -0.10882457 -0.09754378 -0.09141801 -0.08837223 -0.08510112 -0.07798853
 -0.06412148 -0.04212156 -0.01257375  0.02205336  0.05796443  0.09091517
  0.11717669  0.13433866  0.14175571  0.14054304  0.13314834  0.12263588
  0.1118853   0.10291792  0.09651629  0.09221388  0.08862735  0.08401296
  0.07687455  0.06644754  0.05292856  0.0374      0.02148871  0.00687287
 -0.00520992 -0.01430635 -0.02084453 -0.02596171 -0.03111973 -0.03762783
 -0.04621798 -0.05680164 -0.06848259 -0.07982469 -0.08929809 -0.09577406
 -0.09892221 -0.09938977 -0.0987031  -0.09891007 -0.10205581 -0.10963388
 -0.12216431 -0.1

  1.00000000e+00]
class= 1 shape(new5)= (110, 486)	mx= 2.2076714235027834	mn= -4.19601643733877

120 = 110
shape and aug[0]= (481,) [-0.34442631 -0.39821369 -0.42673844 -0.43175682 -0.41730925 -0.38927287
 -0.35466702 -0.32072661 -0.29383798 -0.27979291 -0.27891399 -0.28912621
 -0.30670495 -0.32686096 -0.34476266 -0.35655879 -0.36019242 -0.35581686
 -0.3456965  -0.33359123 -0.32375507 -0.31979113 -0.32366053 -0.33511998
 -0.35175909 -0.36964835 -0.38443431 -0.39258069 -0.39239647 -0.38453755
 -0.37181037 -0.35830658 -0.34810183 -0.34389676 -0.34601686 -0.35210326
 -0.35763571 -0.35718408 -0.34605689 -0.32177901 -0.28478273 -0.24124228
 -0.19833745 -0.16343698 -0.14204056 -0.13616274 -0.14364097 -0.15851622
 -0.17089357 -0.17324587 -0.1608606  -0.13302999 -0.09239064 -0.04763139
 -0.00622197  0.02635803  0.04655178  0.05610428  0.05950081  0.06070929
  0.06277973  0.0659526   0.06854218  0.06779957  0.06245269  0.05366803
  0.04420402  0.03758849  0.03519523  0.03595567  0.03492438  0.0

class= 1 shape(new5)= (110, 486)	mx= 1.5791673802425719	mn= -2.9355319241421336

130 = 110
shape and aug[0]= (481,) [-1.26162697e-01 -9.60912483e-02 -9.41421799e-02 -6.74208581e-02
 -5.59391173e-02 -4.70767349e-02 -3.20362719e-02 -3.04570190e-02
 -3.47637204e-02 -4.46224009e-02 -5.67439002e-02 -7.03035732e-02
 -6.35486653e-02 -9.30254409e-02 -1.00860404e-01 -7.97943198e-02
 -8.48318367e-02 -6.32149738e-02 -6.57696028e-02 -3.03386441e-02
 -1.81386995e-02  6.33750376e-04  2.08040309e-02  3.62877625e-02
  5.08713295e-02  5.37692023e-02  5.78149956e-02  6.05557312e-02
  5.65950945e-02  5.43353492e-02  4.73541143e-02  5.64646593e-02
  7.40004626e-02  7.36955490e-02  1.14717963e-01  1.64236028e-01
  1.47865775e-01  2.00365302e-01  2.20952394e-01  2.26282365e-01
  2.60489437e-01  2.19056076e-01  2.71438840e-01  2.65174696e-01
  2.12945672e-01  2.15972480e-01  1.80338819e-01  2.24442400e-01
  2.51446180e-01  2.52416995e-01  2.26341888e-01  2.76906014e-01
  2.83140600e-01  2.20886674e-01  2.396


140 = 110
shape and aug[0]= (481,) [ 1.28295898e-01  1.50390625e-01  1.61376953e-01  1.61743164e-01
  1.54052734e-01  1.41723633e-01  1.28540039e-01  1.17309570e-01
  1.10290527e-01  1.08154297e-01  1.10351562e-01  1.15478516e-01
  1.21765137e-01  1.27563477e-01  1.31835938e-01  1.34277344e-01
  1.35375977e-01  1.36108398e-01  1.37573242e-01  1.40380859e-01
  1.44775391e-01  1.50390625e-01  1.56372070e-01  1.61376953e-01
  1.64672852e-01  1.65771484e-01  1.65161133e-01  1.63818359e-01
  1.63452148e-01  1.65649414e-01  1.72119141e-01  1.83471680e-01
  1.99462891e-01  2.18994141e-01  2.40112305e-01  2.60498047e-01
  2.78076172e-01  2.91259766e-01  2.99316406e-01  3.03466797e-01
  3.04687500e-01  3.05419922e-01  3.08349609e-01  3.14697266e-01
  3.26171875e-01  3.42773438e-01  3.62792969e-01  3.84521484e-01
  4.05517578e-01  4.23095703e-01  4.35058594e-01  4.40185547e-01
  4.38476562e-01  4.30175781e-01  4.17236328e-01  4.01367188e-01
  3.85009766e-01  3.69873047e-01  3.57421875e-01  3.47

...


210 = 109
shape and aug[0]= (481,) [ 0.24698246  0.2692852   0.2736566   0.26283061  0.24178798  0.21662446
  0.19328453  0.17642993  0.16866723  0.17026733  0.17939455  0.19275205
  0.20646716  0.21700265  0.22189277  0.22016021  0.21235516  0.20024855
  0.18628798  0.17296834  0.1622735   0.15531262  0.15221439  0.15227367
  0.15428293  0.15694184  0.15922968  0.16064616  0.16127042  0.16164102
  0.16250835  0.16454186  0.16808191  0.17300758  0.17875565  0.1844813
  0.18931055  0.19260939  0.19419049  0.19439619  0.194031    0.19415887
  0.19581731  0.19972493  0.20606149  0.21438256  0.2236964   0.2326892
  0.24004587  0.24478808  0.24654444  0.24568276  0.2432665   0.24083945
  0.24008396  0.24242921  0.24869859  0.25887766  0.27205712  0.2865663
  0.30026995  0.31096595  0.3167999   0.31661106  0.31014079  0.29806587
  0.28185755  0.26350282  0.24515092  0.22875868  0.21580259  0.20710561
  0.2027979   0.20240086  0.20499953  0.20945452  0.21460416  0.21941847
  0.22308507  0.22

class= 2 shape(new5)= (111, 486)	mx= 2.267274078608055	mn= -4.404481631913876

220 = 111
shape and aug[0]= (481,) [ 1.09385250e-01  1.26593379e-01  8.12760026e-02  6.55225938e-02
  8.76573358e-02  1.17811030e-01  1.37686574e-01  1.47806184e-01
  1.57813971e-01  1.74358122e-01  1.97383045e-01  2.21551403e-01
  2.41279322e-01  2.53633643e-01  2.60694020e-01  2.66459120e-01
  2.75254804e-01  2.90685913e-01  3.11175954e-01  3.33901758e-01
  3.53481184e-01  3.65358768e-01  3.66328048e-01  3.54677665e-01
  3.30797536e-01  2.99409054e-01  2.64798757e-01  2.31460731e-01
  2.03298365e-01  1.83052443e-01  1.72556416e-01  1.70338297e-01
  1.74513773e-01  1.82480749e-01  1.91420268e-01  1.98755412e-01
  2.02500617e-01  2.01464592e-01  1.95301250e-01  1.84428953e-01
  1.69855356e-01  1.52952066e-01  1.35221286e-01  1.18088141e-01
  1.02740611e-01  9.00268611e-02  8.04095729e-02  7.39697248e-02
  7.04485268e-02  6.93152126e-02  6.98493518e-02  7.12282662e-02
  7.26122963e-02  7.31480415e-02  7.21049

class= 2 shape(new5)= (111, 486)	mx= 1.8503307185817501	mn= -3.1514236952349846

230 = 111
shape and aug[0]= (481,) [ 3.71964150e-01  3.70604653e-01  4.89442276e-01  4.68233753e-01
  3.94238665e-01  4.15861260e-01  3.53001396e-01  3.06612131e-01
  3.14893459e-01  3.39096359e-01  4.06573668e-01  4.40448553e-01
  4.52650396e-01  4.28327443e-01  4.68626280e-01  4.52921344e-01
  4.32855848e-01  4.43102559e-01  4.52255775e-01  4.31778313e-01
  4.32943380e-01  4.58523928e-01  4.44795687e-01  4.93870763e-01
  4.46641952e-01  4.65273198e-01  5.43114672e-01  5.37601102e-01
  6.34995219e-01  4.60218690e-01  5.84131376e-01  4.58178673e-01
  5.73859874e-01  5.10677538e-01  6.11505809e-01  6.89875680e-01
  5.25776862e-01  4.75544233e-01  5.14236262e-01  5.90520808e-01
  5.37083048e-01  6.71484422e-01  5.92196099e-01  7.33128452e-01
  6.61601944e-01  6.84284266e-01  6.13649700e-01  5.85997080e-01
  5.43220651e-01  4.77977554e-01  4.92813962e-01  3.68636245e-01
  3.62570449e-01  4.46095858e-01  4.485

shape and aug[0]= (481,) [ 3.43261719e-01  3.61083984e-01  3.61816406e-01  3.47900391e-01
  3.23486328e-01  2.95166016e-01  2.68310547e-01  2.48413086e-01
  2.38159180e-01  2.38159180e-01  2.46704102e-01  2.60742188e-01
  2.76367188e-01  2.89794922e-01  2.98339844e-01  3.00781250e-01
  2.97607422e-01  2.90283203e-01  2.81250000e-01  2.72949219e-01
  2.67333984e-01  2.65136719e-01  2.66601562e-01  2.70996094e-01
  2.76855469e-01  2.82714844e-01  2.87597656e-01  2.90771484e-01
  2.92968750e-01  2.94189453e-01  2.95410156e-01  2.96875000e-01
  2.98583984e-01  3.00292969e-01  3.01025391e-01  2.99560547e-01
  2.94921875e-01  2.87109375e-01  2.76611328e-01  2.64648438e-01
  2.52929688e-01  2.43652344e-01  2.38403320e-01  2.38159180e-01
  2.42919922e-01  2.51708984e-01  2.62451172e-01  2.72705078e-01
  2.80517578e-01  2.84179688e-01  2.83447266e-01  2.79296875e-01
  2.73681641e-01  2.69531250e-01  2.69531250e-01  2.76367188e-01
  2.91015625e-01  3.13232422e-01  3.41552734e-01  3.73046875e-01


class= 2 shape(new5)= (111, 486)	mx= 2.08203125	mn= -3.390625

250 = 111


In [6]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    aug_pls60,aug_pls61,aug_pls62=3,4,5 #2 is MgWrp and 4 is GDO and 5 is freq Augmentes Data for GAN Training
    vars()['ecg_aug'+str(i)+'60']=np.append(vars()['ecg_aug'+str(i)+'60'],GAN_Gnrt(ecg,i,aug_pls60),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'60'],60,i)
    print(str(i)+'60 =',len(vars()['ecg_aug'+str(i)+'60']))
    vars()['ecg_aug'+str(i)+'61']=np.append(vars()['ecg_aug'+str(i)+'61'],GAN_Gnrt(ecg,i,aug_pls61),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'61'],61,i)
    print(str(i)+'61 =',len(vars()['ecg_aug'+str(i)+'61']))
    #xtrain=np.append(xtrain,vars()['ecg_aug60'+str(i)],axis=0)
    vars()['ecg_aug'+str(i)+'62']=np.append(vars()['ecg_aug'+str(i)+'62'],GAN_Gnrt(ecg,i,aug_pls62),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'62'],62,i)
    print(str(i)+'62 =',len(vars()['ecg_aug'+str(i)+'62']))
    #xtrain=np.append(xtrain,vars()['ecg_aug60'+str(i)],axis=0)




 Class  1  Shape =  (23, 481)
for class  1 number of windows is =  (481,)
GAN Train Data shape= (24, 480)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Optimal cluster number:  5
class= 1 cluster= 0 epochs= 4000  >> shape generate =  (640000, 483)
class= 1 cluster= 1 epochs= 4000  >> shape generate =  (640000, 483)
class= 1 cluster= 2 epochs= 4000  >> shape generate =  (640000, 483)
class= 1 cluster= 3 epochs= 4000  >> shape generate =  (640000, 483)
class= 1 cluster= 4 epochs= 4000  >> shape generate =  (640000, 483)
cls  1  final shape, Generated by GAN = (120, 483)
shape and aug[0]= (481,) [2.109375   2.109375   2.08789062 2.08203125 2.09570312 2.12695312
 2.14648438 2.16210938 2.1640625  2.16601562 2.16210938 2.15820312
 2.15625    2.15820312 2.16210938 2.16601562 2.16796875 2.16992188
 2.16992188 2.16015625 2.1484375  2.13476562 2.125      2.109375
 2.1015625  2.08984375 2.08789062 2.09375    2.09765625 2.10546875
 2.109375   2.11328125 2.11328125 2.11328125 2.11328125 2.109375
 2.10546875 2.09960938 2.09765625 2.09179688 2.08984375 2.0859375
 2.0859375  2.08203125 2.08203125 2.078125   2.07421875 2.0703125
 2.06835938 2.06445312


160 = 120



 Class  1  Shape =  (23, 481)
for class  1 number of windows is =  (481,)
GAN Train Data shape= (24, 480)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Optimal cluster number:  4
class= 2 cluster= 0 epochs= 4000  >> shape generate =  (640000, 483)
class= 2 cluster= 1 epochs= 4000  >> shape generate =  (640000, 483)
class= 2 cluster= 2 epochs= 4000  >> shape generate =  (640000, 483)
class= 2 cluster= 3 epochs= 4000  >> shape generate =  (640000, 483)
cls  2  final shape, Generated by GAN = (120, 483)
shape and aug[0]= (481,) [1.91015625 1.875      1.86230469 1.86816406 1.88378906 1.90136719
 1.92382812 1.94042969 1.95117188 1.95410156 1.94824219 1.94140625
 1.9296875  1.921875   1.91503906 1.91699219 1.92089844 1.93164062
 1.94726562 1.95703125 1.96679688 1.97460938 1.97558594 1.97558594
 1.97070312 1.96972656 1.97167969 1.97949219 1.99023438 2.
 2.01171875 2.02148438 2.02929688 2.03125    2.02929688 2.0234375
 2.015625   2.00976562 2.0078125  2.0078125  2.01171875 2.01757812
 2.02539062 2.03320312 2.0390625  2.04101562 2.0390625  2.03320312
 2.0234375  2.015625   2.0078125  2.00195312 2.         2.00195312
 2.00390625 2.0078125  2.00

class= 2 shape(new5)= (120, 486)	mx= 2.67578125	mn= 1.025390625

261 = 120



 Class  2  Shape =  (23, 481)
for class  2 number of windows is =  (481,)
GAN Train Data shape= (24, 480)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Optimal cluster number:  4
class= 2 cluster= 0 epochs= 4000  >> shape generate =  (640000, 483)
class= 2 cluster= 1 epochs= 4000  >> shape generate =  (640000, 483)
class= 2 cluster= 2 epochs= 4000  >> shape generate =  (640000, 483)
class= 2 cluster= 3 epochs= 4000  >> shape generate =  (640000, 483)
cls  2  final shape, Generated by GAN = (120, 483)
shape and aug[0]= (481,) [1.76171875 1.73144531 1.71386719 1.71191406 1.73242188 1.75878906
 1.7890625  1.8046875  1.80664062 1.80566406 1.79492188 1.78320312
 1.77734375 1.76269531 1.76660156 1.77441406 1.7890625  1.80175781
 1.81445312 1.82617188 1.83398438 1.8359375  1.83398438 1.83007812
 1.82714844 1.82128906 1.82714844 1.83007812 1.84667969 1.85546875
 1.87011719 1.88085938 1.88769531 1.89257812 1.89160156 1.88769531
 1.88085938 1.87109375 1.87402344 1.8671875  1.87402344 1.88378906
 1.89648438 1.89550781 1.90039062 1.90136719 1.90332031 1.89648438
 1.88867188 1.88769531 1.8828125  1.87597656 1.86425781 1.86523438
 1.86132812 1.8574

In [7]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    aug_pls70,aug_pls71,aug_pls72=3,4,5 #2 is MgWrp and 3 is TimeWrp Augmentes Data for LSTM Augmentation
    vars()['ecg_aug'+str(i)+'70']=np.append(vars()['ecg_aug'+str(i)+'70'],LSTM_Gnrt_Pridc(ecg,i,aug_pls70),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'70'],70,i)
    print(str(i)+'70 =',len(vars()['ecg_aug'+str(i)+'70']))
    vars()['ecg_aug'+str(i)+'71']=np.append(vars()['ecg_aug'+str(i)+'71'],LSTM_Gnrt_Pridc(ecg,i,aug_pls71),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'71'],71,i)
    print(str(i)+'71 =',len(vars()['ecg_aug'+str(i)+'71']))
    vars()['ecg_aug'+str(i)+'72']=np.append(vars()['ecg_aug'+str(i)+'72'],LSTM_Gnrt_Pridc(ecg,i,aug_pls72),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'72'],72,i)
    print(str(i)+'72 =',len(vars()['ecg_aug'+str(i)+'72']))

bincount= [ 0 12 11]
LSTM Window size= 120
CNN_LSTM Generation for Class =  1
len_cls  1 = 1
epochs= 120


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


6479/6479 [==============================] - 314s 48ms/step - loss: 17054106.0000 - val_loss: 52705.1914
bs_aug_pls shape= (110, 480)
bs_aug_pls[0]= [-0.34442631 -0.39821369 -0.42673844 -0.43175682 -0.41730925 -0.38927287
 -0.35466702 -0.32072661 -0.29383798 -0.27979291 -0.27891399 -0.28912621
 -0.30670495 -0.32686096 -0.34476266 -0.35655879 -0.36019242 -0.35581686
 -0.3456965  -0.33359123 -0.32375507 -0.31979113 -0.32366053 -0.33511998
 -0.35175909 -0.36964835 -0.38443431 -0.39258069 -0.39239647 -0.38453755
 -0.37181037 -0.35830658 -0.34810183 -0.34389676 -0.34601686 -0.35210326
 -0.35763571 -0.35718408 -0.34605689 -0.32177901 -0.28478273 -0.24124228
 -0.19833745 -0.16343698 -0.14204056 -0.13616274 -0.14364097 -0.15851622
 -0.17089357 -0.17324587 -0.1608606  -0.13302999 -0.09239064 -0.04763139
 -0.00622197  0.02635803  0.04655178  0.05610428  0.05950081  0.06070929
  0.06277973  0.0659526   0.06854218  0.06779957  0.06245269  0.05366803
  0.04420402  0.03758849  0.03519523  0.03595567

class= 1 shape(new5)= (958, 486)	mx= 2.2625532150268555	mn= -3.458148241043091

170 = 958
bincount= [ 0 12 11]
LSTM Window size= 120
CNN_LSTM Generation for Class =  1
len_cls  1 = 1
epochs= 120
6479/6479 [==============================] - 316s 49ms/step - loss: 17053610.0000 - val_loss: 53680.3398
bs_aug_pls shape= (110, 480)
bs_aug_pls[0]= [-1.26162697e-01 -9.60912483e-02 -9.41421799e-02 -6.74208581e-02
 -5.59391173e-02 -4.70767349e-02 -3.20362719e-02 -3.04570190e-02
 -3.47637204e-02 -4.46224009e-02 -5.67439002e-02 -7.03035732e-02
 -6.35486653e-02 -9.30254409e-02 -1.00860404e-01 -7.97943198e-02
 -8.48318367e-02 -6.32149738e-02 -6.57696028e-02 -3.03386441e-02
 -1.81386995e-02  6.33750376e-04  2.08040309e-02  3.62877625e-02
  5.08713295e-02  5.37692023e-02  5.78149956e-02  6.05557312e-02
  5.65950945e-02  5.43353492e-02  4.73541143e-02  5.64646593e-02
  7.40004626e-02  7.36955490e-02  1.14717963e-01  1.64236028e-01
  1.47865775e-01  2.00365302e-01  2.20952394e-01  2.26282365e-01
  2.60

X_prd shape =  (460680, 1, 120, 1)
14397/14397 [==============================] - 231s 16ms/step
augmented shape for class  1 (958, 481)
shape and aug[0]= (481,) [ 5.32799065e-01  5.35355210e-01  5.37751615e-01  5.40040791e-01
  5.42010188e-01  5.43963969e-01  5.45976698e-01  5.48490882e-01
  5.51698089e-01  5.56029201e-01  5.61116278e-01  5.67120552e-01
  5.73151410e-01  5.78957438e-01  5.83879948e-01  5.87156951e-01
  5.88581860e-01  5.88124692e-01  5.86179972e-01  5.83442092e-01
  5.80254197e-01  5.77506661e-01  5.75869858e-01  5.75577438e-01
  5.76908648e-01  5.79233825e-01  5.81434727e-01  5.82865536e-01
  5.82043171e-01  5.78431547e-01  5.72862029e-01  5.66206813e-01
  5.60154736e-01  5.55618942e-01  5.53930223e-01  5.54086924e-01
  5.55190742e-01  5.55483103e-01  5.55151045e-01  5.53174853e-01
  5.48309505e-01  5.40128946e-01  5.28595984e-01  5.14794052e-01
  4.98859316e-01  4.82501179e-01  4.68298674e-01  4.58186001e-01
  4.52168941e-01  4.49216813e-01  4.47848976e-01  4.466866

...

class= 2 shape(new5)= (958, 486)	mx= 1.9868561029434204	mn= -5.669674396514893

270 = 958
bincount= [ 0 12 11]
LSTM Window size= 120
CNN_LSTM Generation for Class =  2
len_cls  2 = 1
epochs= 120
6479/6479 [==============================] - 312s 48ms/step - loss: 2476646144.0000 - val_loss: 7806.7080
bs_aug_pls shape= (111, 480)
bs_aug_pls[0]= [ 3.71964150e-01  3.70604653e-01  4.89442276e-01  4.68233753e-01
  3.94238665e-01  4.15861260e-01  3.53001396e-01  3.06612131e-01
  3.14893459e-01  3.39096359e-01  4.06573668e-01  4.40448553e-01
  4.52650396e-01  4.28327443e-01  4.68626280e-01  4.52921344e-01
  4.32855848e-01  4.43102559e-01  4.52255775e-01  4.31778313e-01
  4.32943380e-01  4.58523928e-01  4.44795687e-01  4.93870763e-01
  4.46641952e-01  4.65273198e-01  5.43114672e-01  5.37601102e-01
  6.34995219e-01  4.60218690e-01  5.84131376e-01  4.58178673e-01
  5.73859874e-01  5.10677538e-01  6.11505809e-01  6.89875680e-01
  5.25776862e-01  4.75544233e-01  5.14236262e-01  5.90520808e-01
  5.3

X_prd shape =  (460680, 1, 120, 1)
14397/14397 [==============================] - 231s 16ms/step
augmented shape for class  2 (958, 481)
shape and aug[0]= (481,) [ 0.36326697  0.3670373   0.3691963   0.37015101  0.37035629  0.37006646
  0.36973     0.36950496  0.36973473  0.37038228  0.37180525  0.37417018
  0.37777606  0.38299215  0.39020637  0.39933071  0.40987864  0.42098391
  0.43150678  0.44025561  0.44619584  0.44873473  0.44810367  0.44492877
  0.44054306  0.43658516  0.43454057  0.43556809  0.44004267  0.44743314
  0.45659271  0.46564123  0.47264639  0.47610673  0.47519758  0.47023106
  0.46254176  0.45429814  0.44790989  0.44552293  0.44855967  0.45732084
  0.47068542  0.4863638   0.50138509  0.51266837  0.51771086  0.515046
  0.50484258  0.48855552  0.46872613  0.44852763  0.43078429  0.41762787
  0.41000962  0.40750578  0.40863875  0.41117701  0.41285577  0.41184649
  0.40726373  0.39926338  0.38899657  0.37816006  0.36853126  0.36138308
  0.35718846  0.35546583  0.35491619 

class= 2 shape(new5)= (958, 486)	mx= 2.1059162616729736	mn= -4.994647979736328

271 = 958
bincount= [ 0 12 11]
LSTM Window size= 120
CNN_LSTM Generation for Class =  2
len_cls  2 = 1
epochs= 120
6479/6479 [==============================] - 313s 48ms/step - loss: 2476633344.0000 - val_loss: 7728.4072
bs_aug_pls shape= (111, 480)
bs_aug_pls[0]= [ 3.43261719e-01  3.61083984e-01  3.61816406e-01  3.47900391e-01
  3.23486328e-01  2.95166016e-01  2.68310547e-01  2.48413086e-01
  2.38159180e-01  2.38159180e-01  2.46704102e-01  2.60742188e-01
  2.76367188e-01  2.89794922e-01  2.98339844e-01  3.00781250e-01
  2.97607422e-01  2.90283203e-01  2.81250000e-01  2.72949219e-01
  2.67333984e-01  2.65136719e-01  2.66601562e-01  2.70996094e-01
  2.76855469e-01  2.82714844e-01  2.87597656e-01  2.90771484e-01
  2.92968750e-01  2.94189453e-01  2.95410156e-01  2.96875000e-01
  2.98583984e-01  3.00292969e-01  3.01025391e-01  2.99560547e-01
  2.94921875e-01  2.87109375e-01  2.76611328e-01  2.64648438e-01
  2.5

X_prd shape =  (460680, 1, 120, 1)
14397/14397 [==============================] - 231s 16ms/step
augmented shape for class  2 (958, 481)
shape and aug[0]= (481,) [ 3.56788307e-01  3.60366970e-01  3.62378806e-01  3.63160819e-01
  3.63183141e-01  3.62842023e-01  3.62435669e-01  3.62198740e-01
  3.62268239e-01  3.62822473e-01  3.64105284e-01  3.66290748e-01
  3.69725734e-01  3.74790013e-01  3.81663173e-01  3.90285581e-01
  4.00145859e-01  4.10443187e-01  4.20246720e-01  4.28316653e-01
  4.33883071e-01  4.36864495e-01  4.36676681e-01  4.33803380e-01
  4.29687053e-01  4.25847918e-01  4.23818409e-01  4.24588621e-01
  4.28710103e-01  4.35712159e-01  4.44276899e-01  4.52663004e-01
  4.59050864e-01  4.62075889e-01  4.61109519e-01  4.56243217e-01
  4.48934525e-01  4.41271812e-01  4.35402840e-01  4.33220625e-01
  4.36012983e-01  4.43962991e-01  4.55893576e-01  4.69713449e-01
  4.82755750e-01  4.92156088e-01  4.95903105e-01  4.93118644e-01
  4.83644247e-01  4.68979776e-01  4.51293707e-01  4.331530

class= 2 shape(new5)= (958, 486)	mx= 2.2357428073883057	mn= -6.461672306060791

272 = 958
